In [119]:
import time
from hashlib import sha256

## Hash

A hash function is one of the most important cryptographic primitives that we will use. It has the following characteristics:

1. **fixed sized**: every input (also called message) creates a hash value of fixed size.
2. **deterministic**: the same input will produce the same output every time.
3. **one-way**: its practically infeasible to invert.
4. **chaotic**: if only one bit changes the whole hash changes in a toatlly chaotic and random way.

There are many differnt hash functions. Here we will use sha256 (used by Bitcoin). It returns 256 bits or 32 bytes. [This](https://emn178.github.io/online-tools/sha256.html) is a great site to look at other hash functions.

#### Create hash digest from string, int or float.

In [ ]:
sha = lambda x: '0x'+sha256(str(x).encode()).hexdigest()

The same input will always produce the same output.

In [ ]:
sha('satoshi')

Changing the input just a little bit changes the whole hash chaotically.

In [ ]:
sha('satochi2')

#### Create random hash for testing.

In [ ]:
rh = lambda: sha(time.time())
h  = rh(); h

#### Turn hash into one of 256 possible emojis. 

In [ ]:
def hash2emoji(h):
    if h[:2]!='0x': h='0x'+h
    offset  = int(h[0:4], 0)
    unicode = b'\U' + b'000' + str(hex(0x1F466+offset))[2:].encode()
    return unicode.decode('unicode_escape')

hash2emoji(h)

#### Print hash in a nice way.

In [ ]:
def ph(h):
    if len(h)<12: return h
    else        : return hash2emoji(h)+' '+h[:12] + '...' + h[-3:]

ph(h)

Print mini hash.

In [ ]:
def pmh(h):
    if len(h)<6: return h
    else       : return hash2emoji(h)+' '+h[:6]

pmh(h)

## Hashable

Objects that are hashable have some common properties.

In [ ]:
class Hashable:
    def __eq__(self, other): return self.hash == other.hash
    def __bytes__(self):     return self.hash.encode()
    def json(self): return json.dumps({**self.__dict__}, indent=4)

    
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        if prop not in ['sig','signed','hash','nonce']: 
                super().__setattr__('hash', sha(self.__dict__))
                
    def __str__(self):
        s = []
        for k,v in self.__dict__.items():
            p = f'{k}:'.ljust(15)
            if hasattr(v,'messageHash'): s.append(p+ph(v.messageHash.hex()))
            elif str(v)[:2]=='0x'      : s.append(p+ph(v))
            elif type(v)   ==float     : s.append(p+str(round(v,8))+' eth')
            else                       : s.append(p+str(v))
        return '\n'.join(s)